In [ ]:
apk = ""
package = ""

if not apk.endswith(".apk"):
    apk = apk + ".apk"
apk_name = apk.replace(".apk", "")

app_decompiled_path = f"{apk_name}_decompiled"

recompiled_apk_path = f"{apk_name}_recompiled.apk"
aligned_apk_path = f"{apk_name}_aligned.apk"
resigned_apk_path = f"{apk_name}_resigned.apk"

!rm -rf {app_decompiled_path}
!apktool d {apk} -o {app_decompiled_path}

In [ ]:
with open (f"{app_decompiled_path}/AndroidManifest.xml", "r", -1, "utf-8") as f:
    data = f.read()
with open (f"{app_decompiled_path}/AndroidManifest.xml", "w+", -1, "utf-8") as f:
    data = data.replace('android:allowBackup="false"', 'android:allowBackup="true"')
    data = data.replace('android:fullBackupContent="false"', '')
    f.write(data)

In [ ]:
from utils.utils import setup_android_path_env

setup_android_path_env()

!rm my_app_final.apk {aligned_apk_path}

!apktool b {app_decompiled_path} -o {recompiled_apk_path}
!zipalign -v 4 {recompiled_apk_path} {aligned_apk_path}

import os
keystore_path = os.path.expanduser("~/.android/debug.keystore")

# 파일 존재 여부 확인
if not os.path.exists(keystore_path):
    raise FileNotFoundError(f"debug.keystore not found at {keystore_path}")

!apksigner sign \
  --ks {debug_keystore_path} \
  --ks-key-alias androiddebugkey \
  --ks-pass pass:android \
  --key-pass pass:android \
  --out {resigned_apk_path} {aligned_apk_path}

!jarsigner -verify {resigned_apk_path}

In [ ]:
if package:
    !adb uninstall {package}
!adb install {resigned_apk_path}

In [ ]:
android_backup_path = f"{apk_name}_backup.ab"
android_backup_tar_path = f"{apk_name}_backup.tar"

In [ ]:
!rm {android_backup_path}
!adb backup -f {android_backup_path} {package}

In [ ]:
import os
import urllib.request

abe_download_url = "https://github.com/nelenkov/android-backup-extractor/releases/download/latest/abe-0059753.jar"

out_dir = "utils"
os.makedirs(out_dir, exist_ok=True)
output_path = os.path.join(out_dir, "abe.jar")

# 파일 다운로드
if not os.path.exists(output_path):
    try:
        print(f"Downloading {abe_download_url}...")
        urllib.request.urlretrieve(abe_download_url, output_path)
        print(f"Downloaded to {output_path}")
    except Exception as e:
        print(f"Failed to download {abe_download_url}: {e}")

!rm {android_backup_tar_path}
!java -jar {output_path} unpack {android_backup_path} {android_backup_tar_path} 1
!tar -xf {android_backup_tar_path}